In [1]:
import os
import sys
sys.path.append('../libraries/') # Append the path within which the user-defined class is in

import joblib
from joblib import Parallel, delayed
import time
import pickle

import autograd.numpy as np
import autograd.numpy.random as npr
npr.seed(0)

from hmmlearn import hmm
import matplotlib.pyplot as plt
import numpy as np
from tkinter import *
from tkinter.filedialog import askopenfilename, askdirectory
from tqdm import tqdm
import scipy.stats as stats

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")
sns.set_context("talk")

import ssm
from ssm.util import find_permutation
from ssm.plots import gradient_cmap, white_to_color_cmap

from match_behavior import match_behavior

## Initialize behavior matcher

In [2]:
data_dir = '../processed_data'
model_dir = '../hmm_models'
behavior_matcher = match_behavior(data_dir, model_dir)

## Load neural data

In [3]:
neural_file_name = 'm1_fr0.1hz_30hz_0s_7200s_0.1s_bins_nooverlap.sav'
behavior_matcher.load_data(neural_file_name, sample_rate=10)

Spike times are binned into shape for hmm: (36000, 46)


## Load behavior

In [5]:
behavior_file_name = 'ag25290_day2_iter13'
behavior_names = ['insignificant', 'insignificant', 'insignificant', 'insignificant', 'rear',
         'insignificant', 'insignificant', 'investigate type 1', 'investigate type 2', 'contra-itch',
         'investigate type 3', 'insignificant', 'insignificant', 'insignificant', 'contra-body groom',
         'face groom type 1', 'dive/scrunch', 'head groom', 'ipsi-orient', 'insignificant',
         'face groom type 2', 'ipsi-body groom', 'ipsi-itch type 1', 'insignificant', 'insignificant', 
         'paw groom', 'locomotion', 'insignificant', 'contra-orient']
behavior_matcher.load_behavior(behavior_file_name, behavior_names, frame_rate=60, delay=6.8503)

mouse stayed in nest for 0.28108749470484795% of time
File #0 (a 6 body parts by 984391 frames) has 30 classes


## Load hmms and identify behaviors

### Merged 16-state hmm

In [11]:
behavior_matcher.load_hmm(K=16, mode='merged')
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0.1, lag_seconds=0.3)

--------------------------------------------------
A 16-state hmm is used to match with behaviors.
--------------------------------------------------
5 unique behaviors identified.
--------------------------------------------------
locomotion: 7
investigate type 2: 5
face groom type 1: 2
contra-itch: 1
contra-body groom: 1


### Merged 32-state hmm

In [7]:
behavior_matcher.load_hmm(K=32, mode='merged')
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0.1, lag_seconds=0.3)

--------------------------------------------------
A 32-state hmm is used to match with behaviors.
--------------------------------------------------
7 unique behaviors identified.
--------------------------------------------------
investigate type 2: 11
locomotion: 9
contra-body groom: 4
face groom type 1: 4
contra-itch: 2
head groom: 1
ipsi-body groom: 1


### Direct 50-state hmm

In [8]:
behavior_matcher.load_hmm(K=50, mode='direct')
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0.1, lag_seconds=0.3)

--------------------------------------------------
A 50-state hmm is used to match with behaviors.
--------------------------------------------------
7 unique behaviors identified.
--------------------------------------------------
investigate type 2: 17
locomotion: 11
contra-itch: 7
face groom type 1: 6
contra-body groom: 4
ipsi-body groom: 4
head groom: 1


### Direct 100-state hmm

In [25]:
behavior_matcher.load_hmm(K=100, mode='direct')
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0.1, lag_seconds=0.3)

--------------------------------------------------
A 100-state hmm is used to match with behaviors.
--------------------------------------------------
10 unique behaviors identified.
--------------------------------------------------
investigate type 2: 31
face groom type 1: 17
locomotion: 17
contra-body groom: 11
contra-itch: 9
head groom: 6
ipsi-body groom: 4
investigate type 1: 3
ipsi-itch type 1: 1
paw groom: 1


### Direct 150-state hmm

In [53]:
behavior_matcher.load_hmm(K=150, mode='direct')
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0.1, lag_seconds=0.3)

--------------------------------------------------
A 150-state hmm is used to match with behaviors.
--------------------------------------------------
11 unique behaviors identified.
--------------------------------------------------
investigate type 2: 46
locomotion: 22
face groom type 1: 21
ipsi-body groom: 21
contra-itch: 13
contra-body groom: 13
head groom: 7
investigate type 1: 4
face groom type 2: 1
ipsi-itch type 1: 1
paw groom: 1


### Direct 200-state hmm

In [55]:
behavior_matcher.load_hmm(K=200, mode='direct')
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0.1, lag_seconds=0.3)

--------------------------------------------------
A 200-state hmm is used to match with behaviors.
--------------------------------------------------
10 unique behaviors identified.
--------------------------------------------------
investigate type 2: 56
face groom type 1: 35
locomotion: 30
contra-itch: 23
ipsi-body groom: 21
head groom: 16
contra-body groom: 15
paw groom: 2
investigate type 1: 1
dive/scrunch: 1


## Match hmm states by a 16-state hmm with behaviors by various leading and laging seconds

In [31]:
behavior_matcher.load_hmm(K=16, mode='merged')

--------------------------------------------------
A 16-state hmm is used to match with behaviors.
--------------------------------------------------


### Counting within 100ms prior to and 300ms post to on-site hmm states

In [32]:
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0.1, lag_seconds=0.3)

5 unique behaviors identified.
--------------------------------------------------
locomotion: 7
investigate type 2: 5
face groom type 1: 2
contra-itch: 1
contra-body groom: 1


### Counting within 100ms prior to and 200ms post to on-site hmm states

In [33]:
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0.1, lag_seconds=0.2)

5 unique behaviors identified.
--------------------------------------------------
investigate type 2: 9
locomotion: 3
face groom type 1: 2
contra-itch: 1
contra-body groom: 1


### Counting within 100ms prior to and 100ms post to on-site hmm states

In [34]:
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0.1, lag_seconds=0.1)

5 unique behaviors identified.
--------------------------------------------------
investigate type 2: 8
locomotion: 4
face groom type 1: 2
contra-itch: 1
contra-body groom: 1


### Counting within right at and 300ms post to on-site hmm states

In [48]:
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0, lag_seconds=0.3)

5 unique behaviors identified.
--------------------------------------------------
investigate type 2: 9
locomotion: 3
face groom type 1: 2
contra-itch: 1
contra-body groom: 1


### Counting within right at and 200ms post to on-site hmm states

In [36]:
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0, lag_seconds=0.2)

5 unique behaviors identified.
--------------------------------------------------
investigate type 2: 9
locomotion: 3
face groom type 1: 2
contra-itch: 1
contra-body groom: 1


### Counting within right at and 100ms post to on-site hmm states

In [49]:
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0, lag_seconds=0.1)

5 unique behaviors identified.
--------------------------------------------------
investigate type 2: 8
locomotion: 4
face groom type 1: 2
contra-itch: 1
contra-body groom: 1


### Counting within 200ms prior to and 300ms post to on-site hmm states

In [50]:
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0.2, lag_seconds=0.3)

5 unique behaviors identified.
--------------------------------------------------
investigate type 2: 8
locomotion: 4
face groom type 1: 2
contra-itch: 1
contra-body groom: 1


### Counting within 200ms prior to and 200ms post to on-site hmm states

In [43]:
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0.2, lag_seconds=0.2)

5 unique behaviors identified.
--------------------------------------------------
investigate type 2: 7
locomotion: 5
face groom type 1: 2
contra-itch: 1
contra-body groom: 1


### Counting within 200ms prior to and 100ms post to on-site hmm states

In [51]:
idt_behav_for_states, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds=0.2, lag_seconds=0.1)

5 unique behaviors identified.
--------------------------------------------------
investigate type 2: 7
locomotion: 5
face groom type 1: 2
contra-itch: 1
contra-body groom: 1
